In [12]:
import numpy as np
import pandas as pd


df = pd.read_csv('/content/tennis.csv')

In [13]:
df.head(14)

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [19]:
from collections import defaultdict

class NaiveBayesScratch:
    def fit(self, X, y):
        self.classes = y.unique()
        self.priors = y.value_counts(normalize=True).to_dict()  # P(class)

        # Likelihoods: P(feature=value | class)
        self.likelihoods = {}
        for feature in X.columns:
            self.likelihoods[feature] = {}
            for c in self.classes:
                subset = X[y == c]
                value_counts = subset[feature].value_counts()
                total = len(subset)
                probs = {val: count/total for val, count in value_counts.items()}
                self.likelihoods[feature][c] = probs

    def predict_single(self, x):
        posteriors = {}
        for c in self.classes:
            posterior = self.priors[c]   # Start with prior
            for feature, value in x.items():
                if value in self.likelihoods[feature][c]:
                    posterior *= self.likelihoods[feature][c][value]
                else:
                    posterior *= 1e-6   # Laplace smoothing
            posteriors[c] = posterior
        return max(posteriors, key=posteriors.get)

    def predict(self, X):
        return [self.predict_single(row) for _, row in X.iterrows()]



In [22]:

X = df.drop(columns=['play'])
y = df['play']

model = NaiveBayesScratch()
model.fit(X, y)

# Predictions
preds = model.predict(X)

accuracy = sum(preds == y) / len(y)
print("Predictions:", preds)
print("Accuracy:", accuracy)


Predictions: ['no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
Accuracy: 0.9285714285714286


In [23]:
new_sample = {'outlook':'rainy', 'temp':'hot', 'humidity':'high', 'windy':False}
print("New Sample Prediction:", model.predict_single(new_sample))


New Sample Prediction: no
